# Aplicación de una RNN para generar texto

Las RNN (Redes Neuronales Recurrentes) están diseñadas para trabajar con datos de secuencia.

Ejemplo de transformación de corpus a datos de entrenamiento para una RNN.

```python
corpus = "Quiero un jugo de naranja fresco"
seq_lenght = 3
```

|        x        |    y    |
|:---------------:|:-------:|
|  Quiero un jugo |    de   |
|    un jugo de   | naranja |
| jugo de naranja |  fresco |

#### Importar Librerias

In [33]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import re
import time
import numpy as np
import pandas as pd
import tensorflow as tf # https://www.tensorflow.org/install

Vamos a crear un generador de texto usando los discursos de Uribe

In [3]:
df = pd.read_csv("../archivos/uribe.csv")
df.head()

,url,titulo,discurso,fecha,lugar
0,http://web.presidencia.gov.co/discursos/discur...,turistica_24022010,“Yo quiero felicitarlos d...,(Bogotá),Febrero 24 de 2010
1,http://web.presidencia.gov.co/discursos/discur...,lloreda_20012010,“Habríamos querido tener ...,(Bogotá),Enero 20 de 2009
2,http://web.presidencia.gov.co/discursos/discur...,bananeros_29042010,"Medellín, 29 abr (SP). “N...",(Medellín),29 de abril de 2010
3,http://web.presidencia.gov.co/discursos/discur...,ccg264_31012010,“Muy apreciados compatrio...,(Bucaramanga),Enero 31 de 2010
4,http://web.presidencia.gov.co/discursos/discur...,audiovisual_21012010,“Una ...,(Bogotá),Enero 21 de 2009


Corpus

In [4]:
text = df.discurso.str.cat(sep=" ")

# RNN

Primero necesitamos vectorizar el texto. Es decir, convertir el corpus en representación numérica.

Para esto creamos dos tablas de búsqueda: de caracteres a números (`char2idx`) y de números a caracteres (`idx2char`)

In [5]:
vocab = sorted(set(text))
print(f'{len(vocab):,.0f} caracteres únicos')

104 caracteres únicos


In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [7]:
# Traducir nuestro texto a números
text_as_int = np.array([char2idx[c] for c in text])

In [37]:
# Ejemplo
np.array([char2idx[c] for c in "Uribe, paraco, Jupyter está berraco"])

array([42, 65, 56, 49, 52,  5,  0, 63, 48, 65, 48, 50, 62,  5,  0, 31, 68,
       63, 72, 67, 52, 65,  0, 52, 66, 67, 87,  0, 49, 52, 65, 65, 48, 50,
       62])

#### La tarea de predicción
Dado un caracter, o una secuencia de caracteres, ¿cuál es el siguiente caracter más probable? 

#### Ejemplo

```python
seq_length = 4

Texto = "Hello"

Input = "Hell"
Output = "ello"
```

**Epoch**: Una epoca es la cantidad de pasos completos en el conjunto de datos de entrenamiento

In [9]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1) # división entera

In [10]:
# Crear el conjunto de datos de entrenamiento
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [12]:
split_input_target("Estas redes neuronales están muy densas")

('Estas redes neuronales están muy densa',
 'stas redes neuronales están muy densas')

In [13]:
dataset = sequences.map(split_input_target)

In [14]:
for i,(input_example, target_example) in  enumerate(dataset.take(2)):
    print("***>>> Example #",i)
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))
    print()

***>>> Example # 0
Input data:  '                    \xa0“Yo quiero felicitarlos de todo corazón por esta nueva  Vitrina Turística que t'
Target data: '                   \xa0“Yo quiero felicitarlos de todo corazón por esta nueva  Vitrina Turística que ta'

***>>> Example # 1
Input data:  'nto ayuda a Colombia. Quiero felicitar a Anato  (Asociación Colombiana de Agencias de Viajes y Turis'
Target data: 'to ayuda a Colombia. Quiero felicitar a Anato  (Asociación Colombiana de Agencias de Viajes y Turism'



In [15]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print(f"Paso {i}")
    print(f"  input: {input_idx} ({repr(idx2char[input_idx])})")
    print(f"  expected output: {target_idx} ({repr(idx2char[target_idx])})")
    print()

Paso 0
  input: 61 ('n')
  expected output: 67 ('t')

Paso 1
  input: 67 ('t')
  expected output: 62 ('o')

Paso 2
  input: 62 ('o')
  expected output: 0 (' ')

Paso 3
  input: 0 (' ')
  expected output: 48 ('a')

Paso 4
  input: 48 ('a')
  expected output: 72 ('y')



#### Crear lotes de entrenamiento
Usaste `tf.data` para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, tienes que mezclar los datos y empaquetarlos en lotes.

**Batch size**: Tamaño del lote. El número de meustras antes de que el modelo sea actualizado.

**Buffer size**: Tamaño del buffer. Evita que se baraje todo el conjunto de datos.

In [16]:
BATCH_SIZE = 256

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((256, 100), (256, 100)), types: (tf.int64, tf.int64)>

#### Crear el modelo
Use `tf.keras.Sequential` para definir el modelo. Este modelo va a tener tres capas:

- `tf.keras.layers.Embedding`: Capa de entrada
- `tf.keras.layers.GRU`: RNN
- `tf.keras.layers.Dense`: Capa de salida

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 512

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [19]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (256, None, 64)           6656      
_________________________________________________________________
gru (GRU)                    (256, None, 512)          887808    
_________________________________________________________________
gru_1 (GRU)                  (256, None, 512)          1575936   
_________________________________________________________________
dense (Dense)                (256, None, 104)          53352     
Total params: 2,523,752
Trainable params: 2,523,752
Non-trainable params: 0
_________________________________________________________________


#### Ensayar el modelo

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 100, 104) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (256, None, 64)           6656      
_________________________________________________________________
gru (GRU)                    (256, None, 512)          887808    
_________________________________________________________________
gru_1 (GRU)                  (256, None, 512)          1575936   
_________________________________________________________________
dense (Dense)                (256, None, 104)          53352     
Total params: 2,523,752
Trainable params: 2,523,752
Non-trainable params: 0
_________________________________________________________________


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
sampled_indices

array([ 97,  61,  34,  91,  70,  21,  33,  24,  30,   2,   2,  20,  54,
        82,   4,  37,  44,  30,  39,  64,  96,  67,  42,  26,  89,  51,
        29,  70,  83,  44,  28,  65,  78,  46,  67,  11,  37,  85, 100,
        48,  53,   1,  63,  73,  29,  89,  31,  58,  43,  47,  89,  46,
        19,  65,  64,  60,  33,  50,   2,  86,  41,  88,  67,  63,  42,
        53,  62,  11,  32,  75,  55,  18,  96,   8,  74,  34,  17,   4,
        90,  24, 102,  57,  15,  73,  52,  72,   0, 101,  73,  36,  75,
         4,  61,  35,  73,  42,   0,  76, 100,   8])

In [24]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 's diciendo ‘mire de dónde venimos, dónde estábamos’, no quiere retroceder  sino que exige de acuerdo'

Next Char Predictions: 
 "—nMíw?LCI'';gÁ)PWIRq–tUEçdHwÉWGr°Yt2PÓ“af!pzHçJkVZçY:rqmLc'ÚTätpUfo2K\xa0h9–/|M8)éC•j6zey ”zO\xa0)nNzU ¡“/"


# Basura

Como podemos observar, el modelo por ahora sólo produce basura. Por eso lo tenemos que entrenar.

In [25]:
# Función de perdida para optimizar el modelo
def loss(labels, logits):
      return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [26]:
model.compile(optimizer='adam', loss=loss)

Entrenar una red neuronal es demorado, por eso debemos ir guardando el progreso

In [43]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints2/'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

Entrenar el modelo
<center><img src='../img/clase14/waiting.jpg' style='height:300px; float: center; margin: 0px 15px 15px 0px'></center>

In [42]:
# Una epoca es la cantidad de pasos completos en el conjunto de datos de entrenamiento
EPOCHS=10

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # Comentar esta linea la primera vez 
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
  7/209 [>.............................] - ETA: 11:35 - loss: 2.0593

KeyboardInterrupt: 

Cargar modelo

In [44]:
p_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
p_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
p_model.build(tf.TensorShape([1, None]))

Hacer predicciones

In [45]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return(start_string + ''.join(text_generated))

In [49]:
print(generate_text(p_model, start_string=u"Trabajar, trabajar y trabajar "))

Trabajar, trabajar y trabajar con la corrupción;   de aparecía una explona comercializar, una disfrencia,  sino porque ahora es la adversación teníamos 500 mil ciudadanos pobres la po: que es necesario de   una plata e inventir en el mundo contra los compatriotas del Gobierno para devel   frustró de la empresa, que el ejercicio de la posibilidad de capturar en economía son muy l. Estamos   en estos 40 días, hemos querido dificultades, solamente estaba en estos de ese eúxilo; por encima del 15   descentralizados; he pedido   capacidad de crédito a la Brigo: nos sentimos mujores que aquí Colombia, ya notorlicio allá.                        El segundo segundo, la pobreza nuestro también, a número de estar calitas en Maya’: el l5 abrimos los esfuerzos que comandantes   odiosas al año, para uno ve los grupos terroristas, para que cussamanos,   secuestrados por el Ferrocarburos:                        ¿Cómo nos habría haciendo Financiero, que no hay unas cosas básicas que hay cuatro   en la 